<a href="https://colab.research.google.com/github/NyusenKyusai/18018535_DataAnalytics/blob/main/TestingDNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I will creating a model for DNN with the dataset that is temperature, dew point, and has all outliers removed.

In [1]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df1 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnTDWeak.csv', index_col=0)
df2 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnTDSnowWeak.csv', index_col=0)
df3 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnTDMaxMinWeak.csv', index_col=0)
df4 = pd.read_csv('https://raw.githubusercontent.com/NyusenKyusai/18018535_DataAnalytics/main/dnnTDMaxMinSndpWeak.csv', index_col=0)

In [2]:
# make sure we have our data by printing it out
print(df1[:6])

   month  year  temp  dewp  NUM_CRIMES
1      1  2015  25.7  16.0         674
2      1  2015  31.5  27.0         648
3      1  2015  29.0  26.1         514
4      1  2015   2.2  -7.4         523
5      1  2015   8.1  -0.5         590
6      1  2015   4.8  -3.9         647


In [323]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df4.iloc[np.random.permutation(len(df4))]

# df1 = 4
# df2 = 5
# df3 = 6
# df4 = 7
# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:7]

# print out the first 6 rows of predictors.
print(predictors[:6])

      month  year  temp  dewp   max   min  sndp
898       7  2017  80.9  66.0  93.0  69.1   0.0
1112      2  2018  19.5  13.4  21.9  15.1  11.8
1703     10  2019  54.6  46.3  63.0  45.0   0.0
249       9  2015  72.0  67.2  84.9  61.0   0.0
1435      1  2019  37.9  31.7  52.0  30.0   0.0
3         1  2015  29.0  26.1  34.0  12.9   2.0


In [324]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,month,year,temp,dewp,max,min,sndp,NUM_CRIMES
898,7,2017,80.9,66.0,93.0,69.1,0.0,758
1112,2,2018,19.5,13.4,21.9,15.1,11.8,537
1703,10,2019,54.6,46.3,63.0,45.0,0.0,723
249,9,2015,72.0,67.2,84.9,61.0,0.0,791
1435,1,2019,37.9,31.7,52.0,30.0,0.0,664


In [325]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,-1]

# print out the first 6 rows of the targets data.
print(targets[:6])

898     758
1112    537
1703    723
249     791
1435    664
3       514
Name: NUM_CRIMES, dtype: int64


In [326]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [327]:
# Split our data into a training set i.e. 75% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.75)
# Split our data into a test set i.e. 20% of the length of the shuffle array
testsize = int(len(shuffle['NUM_CRIMES'])*0.2)
# The final test set size is 100% - 75% - 20% = 5% of the length of the shuffle array.
finaltestsize = len(shuffle['NUM_CRIMES']) - trainsize - testsize

# Define the number of input values (predictors)
# This needs to change with the dataset
nppredictors = 7
# Define the number of output values (targets)
noutputs = 1

In [328]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
# Different Data sets
# dnnTDWeak
# dnnTDSnowWeak
# dnnTDMaxMinWeak
# dnnTDMaxMinSndpWeak
shutil.rmtree('/tmp/dnnTDMaxMinSndpWeak5', ignore_errors=True)

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/dnnTDMaxMinSndpWeak5', hidden_units=[20,18,14], optimizer=tf.train.AdamOptimizer(learning_rate=0.01), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_TRIPS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:testsize + trainsize].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_TRIPS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:testsize + trainsize].values - predslistscale)**2))
print('DNNRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Life Satisfaction Values.
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Life Satisfaction Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:testsize + trainsize] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe8fa11ee10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/dnnTDMaxMinSndpWeak5', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local

Temp Dewp  
Model 1 got a RMSE of 50.8933 | Using the average of 730.8933 it got an RMSE of 74.2303  
Model 2 got a RMSE of 52.1847 | Using the average of 729.3500 it got an RMSE of 73.9598  
Model 3 got a RMSE of 48.6036 | Using the average of 730.7246 it got an RMSE of 74.6153  
Model 4 got a RMSE of 55.4727 | Using the average of 730.0030 it got an RMSE of 76.9963  
Model 5 got an RMSE of 50.9003 | Using the average of 729.5030 it got an RMSE of 74.8641   
  

Temp Dewp Sndp  
Model 1 got a RMSE of 51.3141 | Using the average of 730.9679 it got an RMSE of 78.9202  
Model 2 got a RMSE of 55.0423 | Using the average of 730.8127 it got an RMSE of 80.8340  
Model 3 got a RMSE of 52.2537 | Using the average of 728.5216 it got an RMSE of 73.0606  
Model 4 got a RMSE of 50.6382 | Using the average of 731.3575 it got an RMSE of 75.6402  
Model 5 got an RMSE of 51.0453 | Using the average of 730.5104 it got an RMSE of 76.9885   

Temp Dewp Max Min  
Model 1 got a RMSE of 54.0121 | Using the average of 728.6485 it got an RMSE of 77.2577  
Model 2 got a RMSE of 55.6423 | Using the average of 730.0194 it got an RMSE of 76.3774  
Model 3 got a RMSE of 54.5224 | Using the average of 730.4881 it got an RMSE of 77.0402  
Model 4 got a RMSE of 55.9457 | Using the average of 731.0672 it got an RMSE of 77.3918  
Model 5 got an RMSE of 53.3082 | Using the average of 729.7612 it got an RMSE of 74.2414   

Temp Dewp Max Min Sndp  
Model 1 got a RMSE of 56.2222 | Using the average of 730.1619 it got an RMSE of 80.1506  
Model 2 got a RMSE of 53.0295 | Using the average of 729.9164 it got an RMSE of 76.0492  
Model 3 got a RMSE of 52.8752 | Using the average of 729.5216 it got an RMSE of 77.4147  
Model 4 got a RMSE of 56.1189 | Using the average of 731.8918 it got an RMSE of 79.9833  
Model 5 got an RMSE of 52.0393 | Using the average of 728.2261 it got an RMSE of 74.0608   

In [329]:
# Creating a variable for the last 5% of the data that is easy to handle
input = predictors[testsize + trainsize:]

# Different Data sets
# dnnTDWeak
# dnnTDSnowWeak
# dnnTDMaxMinWeak
# dnnTDMaxMinSndpWeak
# Boilerplate for creating the estimator
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.DNNRegressor(model_dir='/tmp/dnnTDMaxMinSndpWeak5', hidden_units=[20,18,14], enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

# Predicting the values 
preds = estimator.predict(x=input.values)

# Creating a list of predictions 
predslist = preds['scores']*SCALE_NUM_TRIPS

# Creating the RMSE by comparing the targets to the list of predictions
rmse = np.sqrt(np.mean((targets[testsize + trainsize:].values - predslist)**2))

# Calculate the mean of the Number of Crimes Values.
avg = np.mean(shuffle['NUM_CRIMES'][testsize + trainsize:])

print()
# Printing the rmse and average crimes
print('DNNRegression has RMSE of {0}'.format(rmse));
print('Average is {0}'.format(avg))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe8f7a54210>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/dnnTDMaxMinSndpWeak5', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/dnnTDMaxMinSndpWeak5/model.ckpt-10000
INFO:tensorfl

Temp Dewp  
Model 1 got a RMSE of 47.6594  
Model 2 got a RMSE of 45.0424  
Model 3 got a RMSE of 45.6783  
Model 4 got a RMSE of 45.9276  
Model 5 got an RMSE of 51.5507  

Temp Dewp Sndp  
Model 1 got a RMSE of 57.2909  
Model 2 got a RMSE of 48.3006  
Model 3 got a RMSE of 51.9503  
Model 4 got a RMSE of 50.2763  
Model 5 got an RMSE of 47.1352   

Temp Dewp Max Min  
Model 1 got a RMSE of 54.5355  
Model 2 got a RMSE of 54.1363  
Model 3 got a RMSE of 55.5408  
Model 4 got a RMSE of 55.8568  
Model 5 got an RMSE of 49.4387   


Temp Dewp Max Min Sndp  
Model 1 got a RMSE of 47.6700  
Model 2 got a RMSE of 52.4114  
Model 3 got a RMSE of 54.5940  
Model 4 got a RMSE of 58.0916  
Model 5 got an RMSE of 47.5780   